# Application of ResNet18 model on Computed Tomography image dataset

In [ ]:
from tqdm import tqdm
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import torch.utils.data as data
import torchvision
import torchvision.transforms as transforms
import matplotlib.pyplot as plt

import medmnist
from medmnist import INFO, Evaluator
from models import ResNet18

## MedMNIST OrganSMNIST: dataset informations and preprocess

In [ ]:
data_flag = 'organsmnist'

download = True

NUM_EPOCHS = 50
BATCH_SIZE = 128
lr = 0.001

info = INFO[data_flag]
task = info['task']
n_channels = info['n_channels']
n_classes = len(info['label'])

DataClass = getattr(medmnist, info['python_class'])

In [ ]:
# Preprocessing
data_transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean=[.5], std=[.5]) ])
# Loading of the data and split up in to Training, Validation and Test set 
train_dataset = DataClass(split='train', transform=data_transform, download=download)
val_dataset = DataClass(split='val', transform=data_transform, download=download)
test_dataset = DataClass(split='test', transform=data_transform, download=download)
# Data informations
print(train_dataset)
print("===================")
print(test_dataset)
print("===================")
print(val_dataset)

In [ ]:
# Subdivision of the dataset
# Concatenation of the original sets
full_dataset = train_dataset + val_dataset + test_dataset

# New split in Training, Validation and Test set
# Training and provisory sets
train_size = int(0.7 * len(full_dataset))
test_size = len(full_dataset) - train_size
train_dataset, prov_dataset = torch.utils.data.random_split(full_dataset, [train_size, test_size])
# Test and Validation sets
val_size = int(0.3 * len(prov_dataset))
test_size = len(prov_dataset) - val_size
val_dataset, test_dataset = torch.utils.data.random_split(prov_dataset, [val_size, test_size])

pil_dataset = DataClass(split='train', download=download)

# Encapsulate data into dataloader form
train_loader = data.DataLoader(dataset=train_dataset, batch_size=BATCH_SIZE, shuffle=True)
train_loader_at_eval = data.DataLoader(dataset=train_dataset, batch_size=2*BATCH_SIZE, shuffle=False)

val_loader = data.DataLoader(dataset=val_dataset, batch_size=2*BATCH_SIZE, shuffle=False) 

test_loader = data.DataLoader(dataset=test_dataset, batch_size=2*BATCH_SIZE, shuffle=False) 

In [ ]:
# Visualization of some examples
figure = plt.figure(figsize=(10, 10))
cols, rows = 3, 3
for i in range(1, cols * rows + 1):
    sample_idx = torch.randint(len(train_dataset), size=(1,)).item()
    img, label = train_dataset[sample_idx]
    figure.add_subplot(rows, cols, i)
    plt.axis("off")
    plt.imshow(img.squeeze(), cmap="gray")
plt.show()

## Model, loss function and optimizer

In [ ]:
# ResNet18 architecture
model = ResNet18(in_channels=n_channels, num_classes=n_classes).cuda()
    
# Define loss function and optimizer
if task == "multi-label, binary-class":
    criterion = nn.BCEWithLogitsLoss().cuda()
else:
    criterion = nn.CrossEntropyLoss().cuda()
    
optimizer = optim.SGD(model.parameters(), lr=lr, momentum=0.9)

## Functions for the evaluation

In [ ]:
def eval_train_d():
    model.eval().cuda()
    y_true = torch.tensor([]).cuda()
    y_score = torch.tensor([]).cuda()
    
    data_loader = train_loader_at_eval

    with torch.no_grad():
        for inputs, targets in data_loader:
            inputs = inputs.cuda()
            targets = targets.cuda()
            outputs = model(inputs).cuda()

            if task == 'multi-label, binary-class':
                targets = targets.to(torch.float32).cuda()
                outputs = outputs.softmax(dim=-1).cuda()
            else:
                targets = targets.squeeze().long().cuda()
                outputs = outputs.softmax(dim=-1).cuda()
                targets = targets.float().resize_(len(targets), 1).cuda()

            y_true = torch.cat((y_true, targets), 0).cuda()
            y_score = torch.cat((y_score, outputs), 0).cuda()

        y_true = y_true.cpu().numpy()
        y_score = y_score.cpu().detach().numpy()
        ind = np.argmax(y_score, axis = 1)

        B = np.asarray(y_true).flatten()
        acc = ( len(y_true) - len(np.nonzero(B - ind)[0] ) ) / len(y_true)
    
        print('Accuracy training: %.4f' % acc)
    return acc

In [ ]:
def eval_val_d():
    model.eval().cuda()
    y_true = torch.tensor([]).cuda()
    y_score = torch.tensor([]).cuda()
    
    data_loader = val_loader

    with torch.no_grad():
        for inputs, targets in data_loader:
            inputs = inputs.cuda()
            targets = targets.cuda()
            outputs = model(inputs).cuda()

            if task == 'multi-label, binary-class':
                targets = targets.to(torch.float32).cuda()
                outputs = outputs.softmax(dim=-1).cuda()
            else:
                targets = targets.squeeze().long().cuda()
                outputs = outputs.softmax(dim=-1).cuda()
                targets = targets.float().resize_(len(targets), 1).cuda()

            y_true = torch.cat((y_true, targets), 0).cuda()
            y_score = torch.cat((y_score, outputs), 0).cuda()

        y_true = y_true.cpu().numpy() # true labels
        y_score = y_score.cpu().detach().numpy() # probability
        ind = np.argmax(y_score, axis = 1) # index of the max probability
        B = np.asarray(y_true).flatten()

        acc = ( len(y_true) - len(np.nonzero(B - ind)[0]) )  / len(y_true)
    
        print('Accuracy validation: %.4f' % acc)
    return acc

In [ ]:
def eval_test_d():
    model.eval().cuda()
    y_true = torch.tensor([]).cuda()
    y_score = torch.tensor([]).cuda()
    
    data_loader = test_loader

    with torch.no_grad():
        for inputs, targets in data_loader:
            inputs = inputs.cuda()
            targets = targets.cuda()
            outputs = model(inputs).cuda()

            if task == 'multi-label, binary-class':
                targets = targets.to(torch.float32).cuda()
                outputs = outputs.softmax(dim=-1).cuda()
            else:
                targets = targets.squeeze().long().cuda()
                outputs = outputs.softmax(dim=-1).cuda()
                targets = targets.float().resize_(len(targets), 1).cuda()

            y_true = torch.cat((y_true, targets), 0).cuda()
            y_score = torch.cat((y_score, outputs), 0).cuda()

        y_true = y_true.cpu().numpy()
        y_score = y_score.cpu().detach().numpy()
        ind = np.argmax(y_score, axis = 1)

        B = np.asarray(y_true).flatten()
        acc = ( len(y_true) - len(np.nonzero(B - ind)[0]) ) / len(y_true)
    
        print('Accuracy test: %.4f' % acc)
    return acc

## Original ResNet18

In [ ]:
for epoch in range(NUM_EPOCHS):
    print(epoch)
    train_correct = 0
    train_total = 0
    test_correct = 0
    test_total = 0
    
    model.train()
    for inputs, targets in tqdm(train_loader):
        inputs = inputs.cuda()
        targets = targets.cuda()
        # forward + backward + optimize
        optimizer.zero_grad()
        outputs = model(inputs).cuda()
        
        if task == 'multi-label, binary-class':
            targets = targets.to(torch.float32).cuda()
            loss = criterion(outputs, targets).cuda()
        else:
            targets = targets.squeeze().long().cuda()
            loss = criterion(outputs, targets).cuda()
        
        loss.backward()
        optimizer.step()
        

In [ ]:
# Evaluation
print('Evaluating ...')
eval_train_d()
eval_val_d()
eval_test_d()

## Classical Early Stopping (ES)

In [ ]:
acc_val_best = 0
patience = 20
counter = 0

In [ ]:
for epoch in range(NUM_EPOCHS):
    print('Epoch number: %d'% epoch)
    train_correct = 0
    train_total = 0
    test_correct = 0
    test_total = 0
    model.train().cuda()
    for inputs, targets in tqdm(train_loader):
        # forward, backward and optimize
        optimizer.zero_grad()
        inputs = inputs.cuda()
        targets = targets.cuda()
        outputs = model(inputs).cuda()

        if task == 'multi-label, binary-class':
            targets = targets.to(torch.float32).cuda()
            loss = criterion(outputs, targets).cuda()
        else:
            targets = targets.squeeze().long().cuda()
            loss = criterion(outputs, targets).cuda()

        loss.backward()
        optimizer.step()
        
    acc_train = eval_train_d()
    acc_val = eval_val_d()
    acc_test = eval_test_d()
    
    if acc_val > acc_val_best:
        counter = 0
        acc_val_best = acc_val
        model_best = model.train().cuda() 
        print('Counter: %d \n\n'% counter)
    else:
        counter = counter + 1
        print('Counter: %d \n\n'% counter)

    if counter>patience:
        break

## Early Stopping with Learning Rate annealing (LR)

In [ ]:
acc_val_best = 0
patience = 20
counter = 0
lr = 0.001

In [ ]:
for epoch in range(NUM_EPOCHS):
    print('Epoch number: %d \t'% epoch)
    print('LR: %f'% lr)
    train_correct = 0
    train_total = 0
    test_correct = 0
    test_total = 0
    model.train().cuda()
    for inputs, targets in tqdm(train_loader):
        # forward, backward and optimize
        optimizer.zero_grad()
        inputs = inputs.cuda()
        targets = targets.cuda()
        outputs = model(inputs).cuda()

        if task == 'multi-label, binary-class':
            targets = targets.to(torch.float32).cuda()
            loss = criterion(outputs, targets).cuda()
        else:
            targets = targets.squeeze().long().cuda()
            loss = criterion(outputs, targets).cuda()

        loss.backward()
        optimizer.step()
    acc_train = eval_train_d()
    acc_val = eval_val_d()
    acc_test = eval_test_d()
    
    if acc_val > acc_val_best:
        counter = 0
        acc_val_best = acc_val
        model_best = model.train().cuda() 
        print('Counter: %d \n\n'% counter)        
    else:
        counter = counter + 1
        print('Counter: %d \n\n'% counter)
        if counter == patience / 2:
            lr = lr * 0.5

    if counter>patience:
        break

## Early Stopping with Incremental Mini Batch size (BS)

In [ ]:
acc_val_best = 0
patience = 20
counter = 0
lr = 0.001
bs = BATCH_SIZE
batch_inc = 128

In [ ]:
for epoch in range(NUM_EPOCHS):
    print('Epoch number: %d \t'% epoch)
    print('Batch size: %d '% bs)
    train_correct = 0
    train_total = 0
    test_correct = 0
    test_total = 0
    model.train().cuda()
    for inputs, targets in tqdm(data.DataLoader(dataset=train_dataset, batch_size=bs, shuffle=True)):
        # forward + backward + optimize
        optimizer.zero_grad()
        inputs = inputs.cuda()
        targets = targets.cuda()
        outputs = model(inputs).cuda()

        if task == 'multi-label, binary-class':
            targets = targets.to(torch.float32).cuda()
            loss = criterion(outputs, targets).cuda()
        else:
            targets = targets.squeeze().long().cuda()
            loss = criterion(outputs, targets).cuda()

        loss.backward()
        optimizer.step()
    acc_train = eval_train_d()
    acc_val = eval_val_d()
    acc_test = eval_test_d()
    
    if acc_val > acc_val_best:
        counter = 0
        acc_val_best = acc_val
        model_best = model.train().cuda() 
        print('Counter: %d \n\n'% counter)        
    else:
        counter = counter + 1
        print('Counter: %d \n\n'% counter)
        if counter == patience / 2:
            bs = bs + batch_inc

    if counter>patience:
        break

## Early Stopping with Alternate Learning Rate annealing (LR) and Incremental Mini Batch size (LRBS)

In [ ]:
acc_val_best = 0
patience = 20
counter = 0
lr = 0.001
bs = BATCH_SIZE
batch_inc = 128
c2 = 0

In [ ]:
for epoch in range(NUM_EPOCHS):
    print('Epoch number: %d \t'% epoch)
    print('LR: %f '% lr)
    print('Batch size: %d '% bs)
    train_correct = 0
    train_total = 0
    test_correct = 0
    test_total = 0
    model.train().cuda()
    for inputs, targets in tqdm(data.DataLoader(dataset=train_dataset, batch_size=bs, shuffle=True)):
        # forward + backward + optimize
        optimizer.zero_grad()
        inputs = inputs.cuda()
        targets = targets.cuda()
        outputs = model(inputs).cuda()

        if task == 'multi-label, binary-class':
            targets = targets.to(torch.float32).cuda()
            loss = criterion(outputs, targets).cuda()
        else:
            targets = targets.squeeze().long().cuda()
            loss = criterion(outputs, targets).cuda()

        loss.backward()
        optimizer.step()
    acc_train = eval_train_d()
    acc_val = eval_val_d()
    acc_test = eval_test_d()
    
    if acc_val > acc_val_best:
        counter = 0
        acc_val_best = acc_val
        model_best = model.train().cuda() 
        print('Counter: %d \n\n'% counter)        
    else:
        counter = counter + 1
        print('Counter: %d \n\n'% counter)
        if counter == patience / 2:
            c2 = c2 + 1
            if c2%2 == 1:
                lr = lr*0.5
            else:
                bs = bs + batch_inc

    if counter>patience:
        break